# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-121947")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-121947
Azure region: southcentralus
Subscription id: a7e82b92-bbed-4497-a8df-009b71b94f9c
Resource group: aml-quickstarts-121947


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
amlcompute_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    aml_compute = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2',
                                                           max_nodes=4)
    aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
# ws = Workspace.from_config()
# experiment_name = 'your experiment name here'

# experiment=Experiment(ws, experiment_name)

In [4]:
datasetUrl = "https://www.unsw.adfa.edu.au/unsw-canberra-cyber/cybersecurity/ADFA-NB15-Datasets/a%20part%20of%20training%20and%20testing%20set/UNSW_NB15_training-set.csv"

In [12]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.dataset import Dataset

from azureml.core.run import Run




In [ ]:
ds = Dataset.Tabular.from_delimited_files(datasetUrl)

In [5]:
## Function to preprocess the dataset
def clean_data(data):

    x_df = data.to_pandas_dataframe().dropna()

    x_df = x_df.drop(['attack_cat'] , axis=1)
    x_df.service = LabelEncoder().fit_transform(x_df.service)
    x_df.state = LabelEncoder().fit_transform(x_df.state)
    x_df.proto = x_df.proto.replace(x_df.proto.value_counts())
    y_df = x_df.pop("label")
    
    return x_df , y_df

In [13]:
#x, y = clean_data(ds)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.estimator import Estimator
from azureml.train.hyperdrive import choice
import os
import pandas as pd
import numpy as np

# Specify parameter sampler
### YOUR CODE HERE ###


paraSampler = RandomParameterSampling({
    'n_estimators' : choice(range(60 , 70 , 100)),
    'max_depth' : choice( 5, 6 , 7) } )

# Specify a Policy
### YOUR CODE HERE ###

policyBandit = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
 ### YOUR CODE HERE ###

est =  Estimator(source_directory=os.path.join('.'),
                      entry_script='train.py',
                    script_params={'--max_depth': 5,
                        '--n_estimators': 50},
                      compute_target=aml_compute,
                      conda_packages=['pandas==0.23.4',
                                      'scikit-learn==0.21.3',
                                      'numpy'])


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    
                    estimator=est,
                    hyperparameter_sampling=paraSampler,
                    policy=policyBandit,
                    primary_metric_name='Accuracy',
                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                    max_total_runs=3)

In [6]:
exp = Experiment(workspace=ws, name='tuning')

run = exp.submit(hyperdrive_config)
run


WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.
WARNING - If 'arguments' has been provided here and arguments have been specified in 'run_config', 'arguments' provided in ScriptRunConfig initialization will take precedence.
The same input parameter(s) are specified in estimator/run_config script params and HyperDrive parameter space. HyperDrive parameter space definition will override these duplicate entries. ['--max_depth', '--n_estimators'] is the list of overridden parameter(s).


Experiment,Id,Type,Status,Details Page,Docs Page
tuning,HD_d3476862-79f2-4792-ac4e-10b7f83934b6,hyperdrive,Running,Link to Azure Machine Learning studio,Link to Documentation


In [7]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [10]:
best_run= run.get_best_run_by_primary_metric()

In [11]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
tuning,HD_d3476862-79f2-4792-ac4e-10b7f83934b6_1,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [12]:
best_run.get_details()['runDefinition']['arguments']

['--max_depth', '5', '--n_estimators', '60']

In [13]:
## Register the model
best_run= run.get_best_run_by_primary_metric()
model_best_hyper = best_run.register_model(model_name= "model-best-hyper" , model_path= os.path.join('.'))


In [14]:
model_best_hyper

Model(workspace=Workspace.create(name='quick-starts-ws-121947', subscription_id='a7e82b92-bbed-4497-a8df-009b71b94f9c', resource_group='aml-quickstarts-121947'), name=model-best-hyper, id=model-best-hyper:1, version=1, tags={}, properties={})

In [17]:
## Save the model
import joblib
joblib.dump(model_best_hyper, "model.pkl")

            

> # Best model is identified from AutoML experiments and therefore the following steps are provided in AutoMl Notebook

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service